From: https://www.matecdev.com/posts/julia-tutorial-science-engineering.html#google_vignette

Note that there is more array functionality in packages such as LinearAlgebra.jl, StaticArrays.jl, and SparseArrays.jl.

In [1]:
# Direct input
A = [1 2 3; 1 2 4; 2 2 2]     # 3×3 Matrix{Float64}
A = [1 2 3;                   # 3×3 Matrix{Float64}
     1 2 4;
     2 2 2]     
b1 = [4.0, 5, 6]              # 3-element Vector{Float64}
b2 = [4.0; 5; 6]              # 3-element Vector{Float64}
m1 = [4.0 5 6]                # 1×3 Matrix{Float64}
A = ["Hello", 1, 2, 3]        # Vector of type Any

4-element Vector{Any}:
  "Hello"
 1
 2
 3

### Array comprehension vs. generators

A very concise way to initialize and array is by resorting to so-called array comprehensions.
The following code builds an array with the first 100,000 terms of a quadratic series, and then performs the sum.

When using parenthesis instead of the square brackets, Julia will produce a slightly different object: a generator. Generators can be iterated to produce the required values when needed.
As the underlying array isn’t allocated in memory in advance, the generator could have better performance than the array comprehension.

In [2]:
# Array comprehensions
v = [1/n^2 for n=1:100000]
x = sum(v)

1.6449240668982281

In [3]:
# Array generators
gen = (1/n^2 for n=1:100000)
x = sum(gen)

1.6449240668982423

### Undefined arrays
For performance reasons, it is wise to initialize arrays of a given type (and size), without specifying any values.  To do this, we can employ keywords such as Vector{T}, Matrix{T} or Array{T}, where T is a type (see, for example, Numeric Types in the documentation).

The reason to declare undefined arrays is to be able to fill them later — for example, with the use of a for loop.

What happens under the hood when we declare an undef array, is that a certain portion of memory gets reserved (or allocated) for this specific use. As the computer is not even filling that chunk of memory with zeros, we are saving some time.

In [4]:
n = 5
A1 = Array{Float64}(undef,n,n)          # 5×5 Matrix{Float64}
A2 = Matrix{Float64}(undef,n,n)         # 5×5 Matrix{Float64}

V1 = Array{Float64}(undef,n)            # 5-element Vector{Float64}
V2 = Vector{Float64}(undef,n)           # 5-element Vector{Float64}

A = Array{String}(undef,n)
A = Array{Any}(undef,n)

5-element Vector{Any}:
 #undef
 #undef
 #undef
 #undef
 #undef

### Empty arrays
Empty arrays can be a useful starting point when we are in a situation where it is hard or impossible to know which array sizes we need in advance. An empty array can be later grown dynamically, and filled with values. (see the “Dynamic Arrays” section below in this page).

To initialize an empty array, it is perfectly valid to use n = 0 in the above expressions.


To initialize an empty array, it is perfectly valid to use n = 0 in the above expressions. There is a shorthand for this expression: (see second line). A possible source of errors would be to confuse this array with an empty array of “Any” type, which is initialized as follows: (see third line). If we later fill this array dynamically with Float values, its type would remain fixed at “Any”, which could lead to bad performance — or even errors, if we plan on passing this array to a function which requires arrays of type Float, for example.

In [5]:
v = Array{Float64}(undef,0)     
v = Float64[]                # shorthand for above expression
v = []    # Same as Any[], and you can't change this type easily later (gotcha!)

Any[]

### Special kinds of arrays
See Documentation for lots of examples: https://docs.julialang.org/en/v1/manual/arrays/#Construction-and-Initialization

In [6]:
A = zeros(8,9)
B = ones(8,9)
C = rand(6,6)

6×6 Matrix{Float64}:
 0.99907   0.554585  0.671871  0.677795   0.901719  0.998623
 0.134639  0.36978   0.463271  0.089742   0.256451  0.669525
 0.22043   0.954448  0.19185   0.448509   0.715356  0.779611
 0.354458  0.5013    0.679783  0.72332    0.587571  0.240012
 0.620164  0.414974  0.230264  0.0867013  0.190878  0.98802
 0.910155  0.172627  0.631615  0.678909   0.586183  0.875885

In the case of the identity matrix, placing an integer right before a letter ‘I’ will do the job, but this requires the LinearAlgebra package:

In [7]:
using LinearAlgebra
M = 5I + rand(2,2)

2×2 Matrix{Float64}:
 5.2298    0.912095
 0.866389  5.36771

### Applying a scalar function to an array and broadcasting the dot operator
It often happens that we have a function which can be applied to a scalar number, but we want to apply it to an array. Of course, we could resort to a for loop to this element-wise function application. But that would be too much hassle.

Fortunately, in Julia we can easily turn a function than accepts a scalar value, and apply it element-wise to an array. The way to do this is to employ a ‘dot’ after the function’s name.

For example, let’s define a scalar function f, and apply it to an array.

In [8]:
f(x) = 3x^3/(1+x^2)
x = [2π/n for n=1:30]
y = f.(x)

y = sin.(x) # dot operator is also needed for common arithmetic and trigonometric functions
y = 2x.^2 + 3x.^5 - 2x.^8
y = @. 2x^2 + 3x^5 - 2x^8   # We can also broadcast the dot operator with @

30-element Vector{Float64}:
     -4.828671033653536e6
 -18039.263768983117
   -610.7857528421777
    -40.50500091388652
      0.12239075156841928
      3.0788623155395416
      2.5165970179806223
      1.8406756088311296
      1.3594357907060943
      1.0347659706787375
      0.812285884726568
      0.6550760528496817
      0.5403678534183181
      ⋮
      0.23029553231945027
      0.20638290794502417
      0.1861052313390679
      0.16874585207464307
      0.15375911701431397
      0.14072317776024484
      0.12930739401638117
      0.1192493881563927
      0.11033861036842828
      0.10240437795375669
      0.09530704253783058
      0.08893137900051265

### Array indexing

In [9]:
A = rand(6)
A[begin]        # get first element
A[end]          # get last element

0.09005455249867977

### Array slicing
The code below will extract the odd indices in both dimensions from a 6x6 matrix, and store the results in a new 3x3 matrix.

In [10]:
A = rand(6,6)                   # 6×6 Matrix{Float64}
B = A[begin:2:end,begin:2:end]  # 3×3 Matrix{Float64}
C = A[1:2:5,1:2:5]              # Same as B

3×3 Matrix{Float64}:
 0.946132  0.208676  0.115553
 0.137664  0.928805  0.0575288
 0.214493  0.610831  0.227796

### Logical indexing
Let's generate an array of random numbers and set to 0 all the elements with values below 0.5

In [11]:
A = rand(6,6)
display(A)
A[A .< 0.5 ] .= 0
A

6×6 Matrix{Float64}:
 0.436595   0.240957  0.437594  0.0839144  0.272047   0.614452
 0.339334   0.199318  0.322279  0.523422   0.636025   0.863263
 0.249283   0.784279  0.746397  0.7071     0.738009   0.932063
 0.554727   0.212488  0.775327  0.919292   0.514218   0.458519
 0.739399   0.838109  0.242831  0.883338   0.0505889  0.268574
 0.0763766  0.737541  0.832214  0.860269   0.131652   0.670013

6×6 Matrix{Float64}:
 0.0       0.0       0.0       0.0       0.0       0.614452
 0.0       0.0       0.0       0.523422  0.636025  0.863263
 0.0       0.784279  0.746397  0.7071    0.738009  0.932063
 0.554727  0.0       0.775327  0.919292  0.514218  0.0
 0.739399  0.838109  0.0       0.883338  0.0       0.0
 0.0       0.737541  0.832214  0.860269  0.0       0.670013

### Iterating over an Array

In [12]:
# Iterating over single dimensional arrays
A = rand(6)
for i ∈ eachindex(A)
    println(string("i=$(i) A[i]=$(A[i])"))
end

i=1 A[i]=0.3207882529368362
i=2 A[i]=0.11970464000736958
i=3 A[i]=0.22101448554173375
i=4 A[i]=0.21277077770653352
i=5 A[i]=0.7911296437107195
i=6 A[i]=0.7161449857351958


In [13]:
# Iterating over multi-dimensional arrays
A = rand(6,6)
for i ∈ 1:size(A,1), j ∈ 1:size(A,2)
    println(string("i=$(i) j=$(j) A[i,j]=$(A[i,j])"))
end

i=1 j=1 A[i,j]=0.56133814303243
i=1 j=2 A[i,j]=0.478558677953013
i=1 j=3 A[i,j]=0.4441400767892877
i=1 j=4 A[i,j]=0.9795257186210794
i=1 j=5 A[i,j]=0.9356561461749268
i=1 j=6 A[i,j]=0.7140007168347464
i=2 j=1 A[i,j]=0.0657077108866082
i=2 j=2 A[i,j]=0.30899033445765767
i=2 j=3 A[i,j]=0.13572636367454016
i=2 j=4 A[i,j]=0.4132240657697772
i=2 j=5 A[i,j]=0.3491375758849665
i=2 j=6 A[i,j]=0.5350750346392849
i=3 j=1 A[i,j]=0.2501033228678403
i=3 j=2 A[i,j]=0.20559235130682296
i=3 j=3 A[i,j]=0.4698677424567703
i=3 j=4 A[i,j]=0.11378459329816826
i=3 j=5 A[i,j]=0.46111263639024735
i=3 j=6 A[i,j]=0.5698601295488308
i=4 j=1 A[i,j]=0.2148370887210317
i=4 j=2 A[i,j]=0.7885830496338981
i=4 j=3 A[i,j]=0.47842370799932254
i=4 j=4 A[i,j]=0.2645589913954497
i=4 j=5 A[i,j]=0.7023888107482452
i=4 j=6 A[i,j]=0.6723700151599525
i=5 j=1 A[i,j]=0.67761883552081
i=5 j=2 A[i,j]=0.1055045046169486
i=5 j=3 A[i,j]=0.21262522652172955
i=5 j=4 A[i,j]=0.970401390099644
i=5 j=5 A[i,j]=0.1308973387095831
i=5 j=6 A[i,j

In [14]:
# Iterating over multidimensional arrays which may not be 1-indexed
A = rand(6,6)
for i ∈ axes(A,1), j ∈ axes(A,2)
    println(string("i=$i j=$j A[i,j]=$(A[i,j])")) # the parentheses are needed to apply $ to all of the expression A[i,j]
end

i=1 j=1 A[i,j]=0.6052258348539364
i=1 j=2 A[i,j]=0.3511937960639021
i=1 j=3 A[i,j]=0.1868600571870469
i=1 j=4 A[i,j]=0.7986708480117817
i=1 j=5 A[i,j]=0.7656269986380679
i=1 j=6 A[i,j]=0.9916138760512274
i=2 j=1 A[i,j]=0.46622161758255076
i=2 j=2 A[i,j]=0.5442762013505794
i=2 j=3 A[i,j]=0.5813984269166731
i=2 j=4 A[i,j]=0.8070608697692988
i=2 j=5 A[i,j]=0.6817156882199886
i=2 j=6 A[i,j]=0.6572490928829346
i=3 j=1 A[i,j]=0.5304688661205413
i=3 j=2 A[i,j]=0.6179662674201273
i=3 j=3 A[i,j]=0.7633592503658323
i=3 j=4 A[i,j]=0.5726859884365022
i=3 j=5 A[i,j]=0.6748889454031192
i=3 j=6 A[i,j]=0.16743357640470258
i=4 j=1 A[i,j]=0.6008834594066866
i=4 j=2 A[i,j]=0.007872261637169586
i=4 j=3 A[i,j]=0.02106681854312431
i=4 j=4 A[i,j]=0.4857937599022477
i=4 j=5 A[i,j]=0.9229578084336201
i=4 j=6 A[i,j]=0.741715265097302
i=5 j=1 A[i,j]=0.2957832982015841
i=5 j=2 A[i,j]=0.29722578482092066
i=5 j=3 A[i,j]=0.7023291094728119
i=5 j=4 A[i,j]=0.33722414513451127
i=5 j=5 A[i,j]=0.9022711398546437
i=5 j=6 

### Basic operations of multidimensional arrays (i.e. matrix multiplication, dot product)

Matrices (two dimensional arrays) can be multiplied with Matlab-style syntax (* operator). A Matrix and vector can also be multiplied with the (*) operator.

In [15]:
A = [1 2 3; 1 2 4; 2 2 2]
B = [1 2 3; 1 2 4; 2 2 2]   # A*B allowed
v = [4.0, 5, 6]             # A*v allowed
m1 = [4.0 5 6]              # A*m1 not allowed

1×3 Matrix{Float64}:
 4.0  5.0  6.0

### Element-wise multiplication
The dot operator will broadcast the scalar multiplicaion operator on an element-wise fashion.

In [16]:
A.*B

3×3 Matrix{Int64}:
 1  4   9
 1  4  16
 4  4   4

### Dot product

In [17]:
v = rand(1000)
w = rand(1000)
z = dot(v,w)
z = v'w     # Typical linear algebra notation
# I guess ' is transpose?

249.73431007959454

### Backslash operator
Like in Matlab, this is a built-in operator to solve matrices. For square matrices, it will try to solve the linear system. For rectangular matrices, it will see the least squares solution. (WHAT DOES THAT MEAN?)

One GOTCHA you may encounter is that, since 1xN matrices are not the same as N-element vectors, we have to be careful and always employ vectors in the right-hand side of an equation.

In [18]:
b1 = [4.0, 5, 6]        # 3-element Vector{Float64}
b2 = [4.0; 5; 6]        # 3-element Vector{Float64}
m1 = [4.0 5 6]          # 1x3 Matrix{Float64}

x=A\b1                  # Solves A*x=b
x=A\b2                  # Solves A*x=b
#x=A\m1                  # Error!!

3-element Vector{Float64}:
  3.0
 -1.0
  1.0

### Resizing and concatenating arrays
Let's start with default dynamic array operations in Julia. By default, arrays in Julia are Dynamic (e.g. growable, resizable) unless defined otherwise.

In [19]:
# Resizing 1D arrays with Push and Pop
A = Float64[]       # Equivalent to A=Array{Float64}(undef, 0)
push!(A,4)          # Adds the number 4 at the end of the array
push!(A,3)          # Adds the number 3 at the end of the array
v = pop!(A)         # Returns 3 and removes it from A

3.0

In [20]:
pushfirst!(A, 5)
A
v = popfirst!(A)

5.0

In [21]:
A = Float64[]       # Equivalent to A=Array{Float64}(undef, 0)
push!(A,4)
pushfirst!(A, 5)
v = splice!(A, 2)
println(v)
A

4.0


1-element Vector{Float64}:
 5.0

In [22]:
A = Float64[]       # Equivalent to A=Array{Float64}(undef, 0)
push!(A,4)
pushfirst!(A, 5)
deleteat!(A, 2)
A

1-element Vector{Float64}:
 5.0

### Multi-dimensional Array Concatenation

In [23]:
# hcat and vcat work for 2D arrays and below
A = [4 5 6]
B = [6 7 8]

M1 = vcat(A,B)
display(M1)
M2 = hcat(A,B)
display(M2)

# same result can be accomplished with syntax for inputting a matrix
M1 = [A; B]
M2 = [A B]

2×3 Matrix{Int64}:
 4  5  6
 6  7  8

1×6 Matrix{Int64}:
 4  5  6  6  7  8

1×6 Matrix{Int64}:
 4  5  6  6  7  8

In [24]:
# The cat function can also work for n-dimensional arrays
M1 = cat(A, B, dims=1)
display(M1)
M2 = cat(A, B, dims=2)
display(M2)
M3 = cat(A, B, dims=3)
display(M3)

2×3 Matrix{Int64}:
 4  5  6
 6  7  8

1×6 Matrix{Int64}:
 4  5  6  6  7  8

1×3×2 Array{Int64, 3}:
[:, :, 1] =
 4  5  6

[:, :, 2] =
 6  7  8

In [25]:
# Concatenating vectors (different from 1xn matrices!)
a = [1, 2, 3]
b = [4, 5, 6]
# hcat and vcat have the opposite effect from before
v1 = vcat(a, b)
display(v1)

v2 = hcat(a,b)
display(v2)

# equivalently
v1 = [a b] # spaces equivalent to hcat
v2 = [a; b] # semicolons equivalent to vcat

# vector of vectors (concatenate using commas)
v3 = [a,b]
display(v3)

# stack function will concatenate a vector of vectors along a specified dimensional
s1 = stack([a,b], dims=1) # results in 2x3 matrix
display(s1)
s2 = stack([a,b], dims=2) # results in 3x2 matrix
display(s2)

6-element Vector{Int64}:
 1
 2
 3
 4
 5
 6

3×2 Matrix{Int64}:
 1  4
 2  5
 3  6

2-element Vector{Vector{Int64}}:
 [1, 2, 3]
 [4, 5, 6]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

3×2 Matrix{Int64}:
 1  4
 2  5
 3  6

## Functions
Since Julia is JIT compiled, you need to wrap your code inside a function in order to get it compiled. Thus, performance-critical sections of code should always be written inside functions.

In [26]:
# Single line function definition
sum_zeta(s,nterms) = sum(1/n^s for n=1:nterms)

println(sum_zeta(2))

# Multi-line function definition
function sum_zeta(s,nterms)
    x = 0
    for n in 1:nterms
        x = x + (1/n)^s
    end
    return x
end

println(sum_zeta(2, 10000)) # Notice slightly lower precision

LoadError: MethodError: no method matching sum_zeta(::Int64)

[0mClosest candidates are:
[0m  sum_zeta(::Any, [91m::Any[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[26]:2[24m[39m


### Keyword arguments and optional arguments
Keyword arguments are identified by name rather than by position. Their syntax is very similar to optional arguments, with the only difference being that they are placed after a semicolon in the function definition. Notice there can be a slight difference in the precision of your answers depending on whether you enter the value `10000` or `1e6`

In [ ]:
sum_zeta1(s, nterms=10000) = sum(1/n^s for n=1:nterms) # nterms optional argument
sum_zeta2(s; nterms=10000) = sum(1/n^s for n=1:nterms) # nterms keyword argument

println(sum_zeta1(2))
println(sum_zeta1(2, 10000)) # same as above
#println(sum_zeta1(2, nterms=10000)) # error! not a keyword

println(sum_zeta2(2))
#println(sum_zeta2(2, 10000)) # error! need a keyword
println(sum_zeta2(2, nterms=10000))
println(sum_zeta2(2, nterms=1e6)) # notice difference in precision

1.6448340718480652
1.6448340718480652
1.6448340718480652
1.6448340718480652
1.64493306684877


### Functions with multiple outputs
When you have a function that returns multiple outputs, what is happening under the hood is that the function is returning a `tuple` and the `tuple` is being destructured into two variables. We can also call for a single output and destructure is ourselves (see `shape` below). Note that tuples are immutable structures, so we won't be able to modify the values of `shape`, but we can modify the values of `a` and `c`.

In [ ]:
function circle(r)
    area = π * r^2
    circumference = 2π * r
    return area, circumference
end

a1, c1 = circle(1.5) # save outputs destructured as variables
println(a1, "\n", c1)

shape = circle(1.5) # save tuple of outputs, destructure them ourselves
a2, c2 = shape
println(a2, "\n", c2)


7.0685834705770345
9.42477796076938
7.0685834705770345
9.42477796076938


### Functions which modify their input (! notation, aka bang! notation)
It is convention to append an exclamation mark to names of functions that modify their arguments. Note that not every type of variable can be modified by a function when passed as an input: the variable has to be *mutable*. `Arrays` are mutable by default. `Tuples` are not. If you pass an immutable variable type to `x`, the value of the variable `x` will change only inside the function `add_one`, but the process won't affect the value of `x` outside of the function.

Note that bang! notation is also used in the `Plots.jl` visualization library to add more data to an existing `plot` object.

In [ ]:
function add_one!(x)
    x .= x .+ 1
end

x = [1,2,3]
add_one!(x); # x is now [2,3,4]

### Anonymous functions
Sometimes we don't need to assign a name to a function. For instance, if we just need to quickly define a function in order to pass it as an argument to another function, then anonymity can come in handy. 

Suppose we are using the following function `secant` which finds the root of a given function `f` using the secant method. This can be applied to any function `f`. So, we could either define a function for `f` and pass it as an argument, OR we could use anonymous function as shorthand. (That's the `x -> x^2 - x - 1`) below.

In [ ]:
function secant(f,a,b,rtol,maxIters)
    iter = 0
    while abs(b-a) > rtol*abs(b) && iter < maxIters
        c,a = a,b
        b = b + (b-c)/(f(c)/f(b)-1)
        iter = iter + 1
    end
    return b
end

φ = secant( x-> x^2 - x - 1, 1, 2, 1e-15, 10 )

1.6180339887498947